In [1]:
#%matplotlib widget  # equivalent to:
import matplotlib
import ipympl  # this automatically does a "matplotlib.use('module://ipympl.backend_nbagg')"
from matplotlib import pyplot as plt
plt.ion()
from IPython.display import Image

import os
import sys
sys.path.append('/srv/ipython-repo/tj_scripts/cn_plot_style')
import cn_plot_style as cnps

sys.path.append('/srv/ipython-repo/tj_scripts/py_fluoracle')
import py_fluoracle as pyflu


def plot_spectrofluorometer_data(ax, dataset):
        title = dataset['name']
        label = dataset['meta']['Labels']
        _x = dataset['data'][:,0]
        _y = dataset['data'][:,1]
        x = _x[_y != 0]
        y = _y[_y != 0]
        x_label = dataset['meta']['Xaxis']
        y_label = dataset['meta']['Yaxis']

        ln = ax.plot(x, y, label=label)

        #cnps.legend(*lns)

        ax.set_title(title)
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
        return ln

In [3]:
# read in all datasets in a folder
path = '/nfs/home/tobiasj/data/Spectrofluorometer'
datasets = []

for root, dirs, files in os.walk(path):
    if 'data' in root:
        files.sort()
        for file in files:
            if file.endswith('.csv'):
                filepath = os.path.join(root, file)
                dataset = pyflu.load_data(filepath)
                #time = data[:,0] / meta['Step']
                #counts = data[:,1]
                if dataset is not None:
                    datasets.append(dataset)

#datasets[0]

In [5]:
# Create results plot and save them
plt.close('all')

lns = []
with cnps.cn_plot(context='paper', dark=False, usetex=False):
    fig, ax = plt.subplots()
    for dataset in datasets:

        ax.clear()
        ln = plot_spectrofluorometer_data(ax, dataset)
        lns.append(ln)

        #cnps.legend(*lns)
        
        #fig.show()
        
        # Save the figure
        if dataset['dirname'].endswith('data'):
            resultsdir = os.path.relpath(os.path.join(dataset['dirname'], '..', 'results'))
            resultspath = os.path.join(resultsdir, ''.join((dataset['name'], '.png')))
        else:
            resultsdir = os.path.relpath(os.path.join(dataset['dirname'], '..', '..', 'results'))
            prename = os.path.basename(dataset['dirname'].rstrip('/'))
            resultspath = os.path.join(resultsdir, ''.join((prename, '_', dataset['name'], '.png')))
        pyflu.mkdir(resultsdir)
        
        if not os.path.exists(resultspath):
            print('Creating "{}""'.format(resultspath))
            fig.savefig(resultspath)

FigureCanvasNbAgg()

Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample09 - 0min+10nMRad52_like01_02_+2nM1st+1.2nM2nd_anneal_Kinetic Sample1 Scan1.png""
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample09 - 0min+10nMRad52_like01_02_+2nM1st+1.2nM2nd_anneal_Ref Kinetic Sample1 Scan1.png""
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample09 - 0min+10nMRad52_like01_02_+2nM1st+1.2nM2nd_anneal_Trans Kinetic Sample1 Scan1.png""
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample09 - 0min+10nMRad52_like01_02_+2nM1st+1.2nM2nd_anneal_UncorrectedEm Kinetic Sample1 Scan1.png""
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample08_05d - 0min+10nMRad52+2nM1st+1.2nM2nd_anneal_Kinetic Sample1 Scan1.png""
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/results/Sample08_05d - 

In [6]:
# Put all result png files in one overview PDF
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm

path = '/nfs/home/tobiasj/data/Spectrofluorometer'


columns = 2
rows = 5

for root, dirs, files in os.walk(path):
    # look into the results directory
    if root.endswith('results'):
        # Select png files and calculate pages needed for overview pdf
        files = [os.path.join(root, file) for file in files if file.endswith('.png')]
        files = pyflu.humansorted_strings(files)
        
        pages = int(len(files) / (columns*rows)) + 1

        cells = [(p, r, c) for p in range(pages) for r in range(rows) for c in range(columns)]

        # Create overview pdf
        pdfname = os.path.relpath(os.path.join(root, '..', 'overview.pdf'))
        print('Creating "{}"'.format(pdfname))
        c = canvas.Canvas(pdfname, pagesize=A4)
        c.drawString(0, 0, root)
        paper_width, paper_height = A4
        width = paper_width / columns
        height = paper_height / rows

        p = 0
        for file, cell in zip(files, cells):
            page, row, column = cell
            if page > p:
                c.drawString(0, 0, root)
                p += 1
                c.showPage()
            c.drawImage(file, width * column, paper_height - height - (height * row), width, height)

        c.drawString(0, 0, root)
        c.save()

Creating "../../../data/Spectrofluorometer/2018-06-27 - annealing with Rad52/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-16 - annealing MA oligos 10 nM Rad52/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-16 - binding atto 636 1B-5B/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-16 - repeating alexa_atto_aneal/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-15 - annealing and binding with Rad52/overview.pdf"
Creating "../../../data/Spectrofluorometer/2016-01-21 - Atto 488/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-14 - annealing_1nM/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-07-04 - binding and annealing test/overview.pdf"
Creating "../../../data/Spectrofluorometer/2016-01-21 - Atto 633/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-07-14 - binding Rad52 diff conc/overview.pdf"
Creating "../../../data/Spectrofluorometer/2018-06-14 - em_ex_scans - alexa488nM/

In [92]:
dsubsets = [dataset
            for dataset
            in datasets
            if '2018-07-04' in dataset['dirname']]

dsubsets = pyflu.humansorted_datasets(dsubsets)

dsets = []
for dset in dsubsets:
    if 'Sample18' in dset['name'] or 'Sample19' in dset['name']:
        dsets.append(dset)

# Plot individual dataset
plt.close('all')
lns = []
with cnps.cn_plot(context='paper', dark=False, usetex=False) as cnp:
    fig, ax = plt.subplots()

    ax.clear()
    for dataset in dsets:
        data = dataset['data'][:,1]
        Fmax = data.max()
        #Fmax = max_fluorescence(data)
        start, stop = None, None
        start, stop = decay_region(data, F0=Fmax, dilution=1)
        x = dataset['data'][:,0][start:stop]
        y = dataset['data'][:,1][start:stop]
        
        # Set first datapoint to time 0 + 30 s (~ duration needed for mixing)
        x = x - x[0] + 30
        
        # select the first seconds
        #duration = 13000  # s
        #x = x[:int(duration / 10)]
        #y = y[:int(duration / 10)]
        
        # get the label for this dataset
        label = dataset['name'].replace('_', ' ').replace('1st', '').replace('2nd', '')
        #label = dataset['meta']['Labels']
                
        # get the color for this dataset
        c = cnp.color

        # Plot the data 2nd the fit
        lns.append(ax.plot(x[::], y[::] / Fmax, '.', label=label, color=c))
        
        cnps.legend(*lns)

        #title = 'Annealing with and without Rad52'
        x_label = dataset['meta']['Xaxis']
        #y_label = dataset['meta']['Yaxis']
        y_label = 'Reactive ssDNA (%)'

        #ax.set_title(title)
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)

    fig.show()

/usr/lib/python3/dist-packages/matplotlib/__init__.py:943: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [ ]:
filename = '../BLABLABLUB.png'
fig.savefig(filename)
plt.close(fig)
Image(filename)